# Curation Estaciones Meteorologicas

Estudiar La cantidad de datos validos disponibles para cada variable en cada estacion y obtener el numero de estaciones con los datos minimos necesarios para el estudio

In [1]:
setwd("~/Repositories/AirQualityCOVID")

In [2]:
source("src/general.R")

## Funciones para filtrar datos

In [3]:
suppressMessages(library(lubridate))
suppressMessages(library(tidyverse))
suppressMessages(library(plyr))

In [4]:
get.info <- function(st) {
    
    dataframe <- read.csv(paste("data/Curation/AEMET/Values/",
                                st,
                                ".csv", sep=""))
    
    dataframe$fecha <- ymd(dataframe$fecha)
    period <- as.numeric(max(dataframe$fecha)-min(dataframe$fecha)) +2
    
    cbind(data.frame(site=st,
                     start_dt=min(dataframe$fecha),
                     end_dt=max(dataframe$fecha),
                     miss_dy=period - nrow(dataframe)
                    ),
          rbind(apply(dataframe, 2, 
                      function(df, period) {
                          (sum(!is.na(df)) / period)
                      }, period))
         )
}

In [5]:
sitesAQ <- read.csv("data/Curation/sitesAQ.csv",
                    stringsAsFactor=TRUE)

all.data <- do.call(rbind.fill, lapply(levels(sitesAQ$site), get.info))

# Guardar Datos

In [6]:
#```R
write.csv(all.data,
          "data/Curation/AEMET/info_sites.csv",
          row.names=F)
#```

In [7]:
info <- data.frame()
percent <- c(0.5, 0.7, 0.8, 0.9)

for (pr in percent) {
    info <- rbind(info,
                  rbind(apply(all.data, 2, 
                              function(df){
                                  sum(df>= pr, na.rm=TRUE)
                              }))[10:ncol(all.data)]
                 )
}

names(info) <- names(all.data)[10:ncol(all.data)]
row.names(info) <- percent

In [8]:
info

,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin,siteAQ
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0.5,67,67,67,62,67,62,54,55,54,60,51,56,56,56,56,67
0.7,67,67,67,62,67,62,54,54,54,60,50,56,56,56,56,67
0.8,66,67,66,62,66,62,54,54,54,60,49,56,56,56,56,67
0.9,61,67,61,62,61,62,54,54,54,60,47,56,56,56,56,67
